In [1]:
from qiskit import QuantumCircuit, transpile, execute, IBMQ
from states import Key, Ciphertext
from global_parameters import GlobalParameters
from encryption_circuit import encrypt
from verification_circuit import verify_deletion_counts
from decryption_circuit import create_decryption_circuit, decrypt_results
from deletion_circuit import delete
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeMontreal

In [2]:
# Qiskit parameters
provider = IBMQ.load_account()
noise_model = FakeMontreal()
backend = AerSimulator.from_backend(noise_model)
# backend = provider.get_backend("ibmq_santiago")
optimization_level = 1
shots = 1000

In [3]:
# Utility function to run a circuit, and return the result counts
def run_and_measure(circuit: QuantumCircuit) -> dict[str, int]:
    print(circuit.draw())
    transpiled_circuit = transpile(
        circuit, backend=backend, optimization_level=0)
    # result = execute(transpiled_circuit, backend=backend,
    #                  optimization_level=optimization_level, shots=1).result()
    result = backend.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()
    # Reverse the string since the most significant qubit is at the 0th index of the measurement string
    reversed_keys_dict = {}
    for key, value in counts.items():
        reversed_keys_dict[key[::-1]] = value
    return reversed_keys_dict

In [4]:
# Encrypt 
global_params = GlobalParameters(1)
key = Key.generate_key(global_params)
message = "001011"
ciphertext = encrypt(message, key, global_params)
original_circuit = ciphertext.circuit.copy() # We might modify this circuit in subsequent steps

In [5]:
# Test 1 - delete
deletion_circuit = delete(ciphertext)
deletion_counts = run_and_measure(deletion_circuit)
verify_deletion_counts(deletion_counts, key)

         ┌───┐      ░ ┌───┐ ░ ┌─┐                                          
    q_0: ┤ X ├──────░─┤ H ├─░─┤M├──────────────────────────────────────────
         ├───┤      ░ ├───┤ ░ └╥┘┌─┐                                       
    q_1: ┤ X ├──────░─┤ H ├─░──╫─┤M├───────────────────────────────────────
         ├───┤      ░ ├───┤ ░  ║ └╥┘┌─┐                                    
    q_2: ┤ H ├──────░─┤ H ├─░──╫──╫─┤M├────────────────────────────────────
         └───┘      ░ ├───┤ ░  ║  ║ └╥┘┌─┐                                 
    q_3: ───────────░─┤ H ├─░──╫──╫──╫─┤M├─────────────────────────────────
                    ░ ├───┤ ░  ║  ║  ║ └╥┘┌─┐                              
    q_4: ───────────░─┤ H ├─░──╫──╫──╫──╫─┤M├──────────────────────────────
                    ░ ├───┤ ░  ║  ║  ║  ║ └╥┘┌─┐                           
    q_5: ───────────░─┤ H ├─░──╫──╫──╫──╫──╫─┤M├───────────────────────────
         ┌───┐┌───┐ ░ ├───┤ ░  ║  ║  ║  ║  ║ └╥┘┌─┐                        
    q_6: ┤ X

In [6]:
# Test 2 - decrypt
decryption_circuit = create_decryption_circuit(key, ciphertext)
decryption_counts = run_and_measure(decryption_circuit)
decrypt_results(decryption_counts, key, ciphertext, message)

         ┌───┐      ░       ░ ┌─┐                                          
    q_0: ┤ X ├──────░───────░─┤M├──────────────────────────────────────────
         ├───┤      ░       ░ └╥┘┌─┐                                       
    q_1: ┤ X ├──────░───────░──╫─┤M├───────────────────────────────────────
         ├───┤      ░ ┌───┐ ░  ║ └╥┘┌─┐                                    
    q_2: ┤ H ├──────░─┤ H ├─░──╫──╫─┤M├────────────────────────────────────
         └───┘      ░ └───┘ ░  ║  ║ └╥┘┌─┐                                 
    q_3: ───────────░───────░──╫──╫──╫─┤M├─────────────────────────────────
                    ░       ░  ║  ║  ║ └╥┘┌─┐                              
    q_4: ───────────░───────░──╫──╫──╫──╫─┤M├──────────────────────────────
                    ░       ░  ║  ║  ║  ║ └╥┘┌─┐                           
    q_5: ───────────░───────░──╫──╫──╫──╫──╫─┤M├───────────────────────────
         ┌───┐┌───┐ ░ ┌───┐ ░  ║  ║  ║  ║  ║ └╥┘┌─┐                        
    q_6: ┤ X

In [7]:
# Test 3 - delete and then decrypt
deletion_circuit = delete(ciphertext)
ciphertext.circuit = deletion_circuit # To use the new circuit as the starting point of the decryption circuit
decryption_circuit = create_decryption_circuit(key, ciphertext)
counts = run_and_measure(decryption_circuit)
deletion_counts = {}
decryption_counts = {}
for measurement, count in counts.items():
    # Qiskit will return a space-separated string of the two measurements. 
    # run_and_measure will reverse the string so that the first substring is the first measurement.
    deletion, decryption = measurement.split(" ")
    deletion_counts[deletion] = deletion_counts.get(deletion, 0) + count
    decryption_counts[decryption] = decryption_counts.get(decryption, 0) + count
verify_deletion_counts(deletion_counts, key)
decrypt_results(decryption_counts, key, ciphertext, message)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

          ┌───┐      ░ ┌───┐ ░ ┌─┐                                           ░ »
     q_0: ┤ X ├──────░─┤ H ├─░─┤M├───────────────────────────────────────────░─»
          ├───┤      ░ ├───┤ ░ └╥┘┌─┐                                        ░ »
     q_1: ┤ X ├──────░─┤ H ├─░──╫─┤M├────────────────────────────────────────░─»
          ├───┤      ░ ├───┤ ░  ║ └╥┘┌─┐                                     ░ »
     q_2: ┤ H ├──────░─┤ H ├─░──╫──╫─┤M├─────────────────────────────────────░─»
          └───┘      ░ ├───┤ ░  ║  ║ └╥┘┌─┐                                  ░ »
     q_3: ───────────░─┤ H ├─░──╫──╫──╫─┤M├──────────────────────────────────░─»
                     ░ ├───┤ ░  ║  ║  ║ └╥┘┌─┐                               ░ »
     q_4: ───────────░─┤ H ├─░──╫──╫──╫──╫─┤M├───────────────────────────────░─»
                     ░ ├───┤ ░  ║  ║  ║  ║ └╥┘┌─┐                            ░ »
     q_5: ───────────░─┤ H ├─░──╫──╫──╫──╫──╫─┤M├────────────────────────────░─»
          ┌───┐┌───┐ ░ ├───┤